<a href="https://colab.research.google.com/github/supercurlyman/praktikum.yandex/blob/master/ideal_borrower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект «Предобработка данных»

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Открыть файл с данными и изучите общую информацию. 
<a id="ы"></a>

In [ ]:
import pandas as pd
credit = pd.read_csv('/datasets/data.csv')

In [ ]:
credit

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Наблюдения
Столбцы `days_employed` и `total_income` имеют лишние знаки после запятой


### Получим сведенья датафрейма

In [ ]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Наблюдения
Столбцы `days_employed` и `total_income` имеют пропуски      

### Использую метод описательной статистики для поиска всех числовых аномалий.

In [ ]:
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Наблюдения

1. Столбец `children` отрицательное минимальное значение((техническая ошибка? Пока не знаю причин по которым такое могло произойти)
2. Столбец `children` высокое максимальное значение(необходимо проверить на ошибку)
3. Столбец `days_employed` отрицательные минимальное значение (техническая ошибка? Пока не знаю причин по которым такое могло произойти)
4. Столбец `ys_employed` завышенно максимальное значение (больше возможной продолжительности жизни)
5. Столбец `ob_years` минимальное значение 0(должно быть > 18)

### Проведем поиск полных дубликатов, сгрупируем их по максимально вариабильным столбцам

In [ ]:
print(credit.duplicated().sum())
credit[credit.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,NaN,46,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,высшее образование
520,0,NaN,35,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу
541,0,NaN,57,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,сделка с подержанным автомобилем
554,0,NaN,60,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,покупка недвижимости
680,1,NaN,30,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости


### Наблюдения  
- 54 Дубликата, в целом не так много, но лучше удалить

### Проверим вариабильность строковых данных таблицы

### Вывод
1. Имеются пропуски данных и нулевые значения в столбцах `days_employed1`, `total_income`, `dob_years`. Необходимо будет определить закономерность пропусков и нулевых значений, возможную связь. Определить метод замены пропуска.
2. Необходимо выяснить закономерность и причины отрицательных значений данных.
3. Проверить корректность указанного большого количества детей в `children`
4. Исправить явно завышенные данные стажа в столбце `days_employed`
5. Исправить дубликаты `educatio` и `purpose` методом лемматизации и выравниванием регистра
6. Удалить полные дубликаты
7. Данные пола полностью случайны, можно подставить любой.

После исправление всех ошибок приступать к категоризации и анализу данных.

### Шаг 2. Предобработка данных
<a id="preprocessing"></a>

### Обработка некоректных данных

1. Первая аномалия 'children' это отрицательное количество детей, не могу предположить причины таких данных. Исправлю по модулю
2. Вторая аномалия 'children' это максимальное количество 20 детей, проверю методом уникальных значений 
Кроме числа 20, более 5 детей в столбце не встречается. 
Предполагаю что 20 детей это ошибка, 0 мог быть ошибкой ввода(находится рядом на numpad, возможно 0 стоял по дефолту в строке ввода количества детей).  
Заменю все значения 20 на 2

In [ ]:
credit['children'] = credit['children'].abs()
print(credit['children'].value_counts())
credit.loc[credit["children"] == 20,"children"] = 2

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Отрицательные значения `days_employed` не могу предположить причины таких данных. Исправлю по модулю

In [ ]:
credit['days_employed'] = credit['days_employed'].abs()

- Максимальное количество отработанных дней является ошибкой.  
Напишем функцию, которая будет искать повышенное значение отработанных дней и заменять их на медиану стажа данного возраста.  
В случае если большинство показателей стажа данного возраста завышенны, тогда заменять на максимально возможный стаж по возрасту.

In [ ]:
def err_days_employed(row):
    if row['days_employed'] > (row['dob_years'] - 16) * 365:
        if credit[credit['dob_years'] == row['dob_years']]['days_employed'].median() < (row['dob_years'] - 16) * 365:
            return credit[credit['dob_years'] == row['dob_years']]['days_employed'].median()
        else: 
            return (row['dob_years'] - 16) * 365
    else:
        return row['days_employed']
credit['days_employed'] = credit.apply(err_days_employed, axis=1)

### Обработка пропусков
<a id="null"></a>

In [ ]:
credit_null = (credit[credit['days_employed'].isnull()]).copy(deep=True) #Методом isnull() выведем пропуски по столбцу 'days_employed'
credit_null

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


- Прослеживается закономерность в пропусках данных 'days_employed' и 'total_income'  
Проверим, на закономерность пропуска

In [ ]:
credit_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


- дейсчтвительно, все строки с нулевыми значениями пересекаются   
Проверим, может ли это быть связано со статусом безработного или пенсионера

In [ ]:
print(credit_null['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


- Нет пропуски имеются в различных категориях занятости  
Предположу, что отсутствие данных связано с непредоставлением справки с места работы
Буду заменять пропуски в категории `days_employed` средним по возрасту.  
В категории `total_income` средним по категории занятости.

In [ ]:
credit["days_employed"].fillna(-1, inplace = True) #заменил пропуски стажа на -1 чтобы с помощью функции заменить на медиану значений

In [ ]:
credit["total_income"].fillna(-1, inplace = True) #заменил пропуски зарплаты на -1 чтобы с помощью функции заменить на медиану значений

In [ ]:
def non_total_income(row): # пишу функцию для замены пропусков стажа медианой стажа в соответствии с возрастом
    if row['days_employed'] == -1:
        return credit[credit['dob_years'] == row['dob_years']]['total_income'].median()
    else:
        return row['days_employed']
credit['days_employed'] = credit.apply(non_total_income, axis=1)

In [ ]:
def non_days_employed(row): # пишу функцию для замены пропусков зарплаты медианой зарплаты в соответствии с категорией занятости
    if row['days_employed'] == -1:
        return credit[credit['income_type'] == row['income_type']]['total_income'].median()
    else:
        return row['total_income']
credit['total_income'] = credit.apply(non_days_employed, axis=1)
# credit.info()

Удалю все строки с отсутствием возраста просто потому что их меньше 0,5%, а методом dropna для удаления пропусков не удалось воспользоваться

In [ ]:
credit = credit[credit['dob_years'] != 0]

Удалю строчку с полом 'XNA', заметил days_employed и total_income одно и тоже число, что явно говорит о некоректности данных.       

In [ ]:
credit = credit[credit['gender'] != 'XNA']

### Вывод

Очень важно знать каким образом формируются данные, для того чтобы найти подходящий метод замены случайных и не случайных пропусков. Полностью случайные пропуски в этой практической работе не влияют на анализ. 

### Замена типа данных

Зарплату `total_income` и стаж в днях `days_employed` переводим из 'float64' в 'int'. Данные после запятой на результат.
Используем функцию 'astype'.

In [ ]:
credit['days_employed'] = credit['days_employed'].astype('int')
credit['total_income'] = credit['total_income'].astype('int')
#credit.dtypes #Проверим типы данных столбцов

### Вывод

Замена типов данных в нашем случае улучшает визуальное восприятие, на больших таблицам сможет сэкономить вычислительные ресурсы. Если бы тип данных был строковый то возможно воспользоваться функцией 'to_numeric()'.

### Обработка дубликатов

Удаляем дубликаты, проверяем.

In [ ]:
credit = credit.drop_duplicates()
credit.duplicated().sum()

0

### Вывод

Причины задвоения данных в отличии от пропуска данных не столь важны. В нашем случае их не так много, но если их будет больше, это может вызвать смещение результатов. Поэтому лучше удалить.

### Лемматизация

В столбце `purpose`содержатся похожие друг на друга цели, описанные разными словами. Катетеризируем цели с помощью лемматизации.

Загрузим библиотеки необходимые для лемматизации. Снова выведем список уникальных целей

In [ ]:
from pymystem3 import Mystem
m = Mystem()
from nltk.stem import SnowballStemmer 
credit['purpose'].value_counts()

свадьба                                   788
на проведение свадьбы                     769
сыграть свадьбу                           764
операции с недвижимостью                  672
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     648
операции с жильем                         646
жилье                                     640
покупка жилья                             640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          622
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      617
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

1. Создадим список лемм
2. Выведем его по популярности

In [ ]:
list_of_lemmas = [] 
for element in credit['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)
from collections import Counter
unique_lemmas = Counter(list_of_lemmas) #посчитаем леммы 
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True) #отсартируем


[(' ', 33460),
 ('\n', 21369),
 ('недвижимость', 6329),
 ('покупка', 5872),
 ('жилье', 4437),
 ('автомобиль', 4286),
 ('образование', 3996),
 ('с', 2904),
 ('операция', 2593),
 ('свадьба', 2321),
 ('свой', 2224),
 ('на', 2216),
 ('строительство', 1874),
 ('высокий', 1366),
 ('получение', 1310),
 ('коммерческий', 1307),
 ('для', 1287),
 ('жилой', 1225),
 ('сделка', 938),
 ('дополнительный', 903),
 ('заниматься', 900),
 ('проведение', 769),
 ('сыграть', 764),
 ('сдача', 650),
 ('семья', 637),
 ('собственный', 633),
 ('со', 627),
 ('ремонт', 605),
 ('подержанный', 484),
 ('подержать', 478),
 ('приобретение', 459),
 ('профильный', 435)]

In [ ]:
lemmas_word = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'] #выделил основные леммы не связаные с подлезжащими и глаголами

- Напишем функцию для категоризации по определенным леммам

In [ ]:
def purpose_group(row):
    lema = m.lemmatize(row)
    if 'автомобиль' in lema:
        return 'автомобиль'
    if 'недвижимость' in lema:
        return 'недвижимость'
    if 'жилье' in lema:
        return 'недвижимость'
    if 'образование' in lema:
        return 'образование'
    if 'свадьба' in lema:
        return 'свадьба'
credit['purpose_lemm'] = credit['purpose'].apply(purpose_group)


In [ ]:
credit['education'] = credit['education'].str.lower() # переведем 'education' в нижний регистр
credit

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,3650,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21521,0,18615,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


### Вывод

Мы получили список целей получения кредита: 'недвижимость','автомобиль', 'образование', 'свадьба'

## Категоризация данных

- Выше мы получили категории целей кредита
- Проверим на ошибки уже имеющиеся категории 'education'-'education_id' и 'family_status'-'family_status_id'

In [ ]:
credit.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5216
1,начальное,3,282
2,неоконченное высшее,2,741
3,среднее,1,15124
4,ученая степень,4,6


In [ ]:
credit.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2794
1,в разводе,3,1185
2,вдовец / вдова,2,954
3,гражданский брак,1,4141
4,женат / замужем,0,12295


Разобьем на уровни дохода `total_income` Для этого воспользуемся квантилями 25%, 50% и 75%.


In [ ]:
quan_total_income = credit['total_income'].describe()
quan_total_income

count    2.136900e+04
mean     1.509273e+05
std      1.098811e+05
min     -1.000000e+00
25%      8.895100e+04
50%      1.357180e+05
75%      1.957370e+05
max      2.265604e+06
Name: total_income, dtype: float64

Создадим функцию для присваивания номера определенной  долли по возрастанию дохода

In [ ]:
def income_group_fun(row): 
    if row <= quan_total_income[4]:
        return 1
    elif quan_total_income[4] < row <= quan_total_income[5]: 
        return 2
    elif quan_total_income[5] < row <= quan_total_income[6]: 
        return 3
    else:
        return 4


credit['income_group'] = credit['total_income'].apply(income_group_fun) 
credit #проверю правильность работы функции

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4
4,0,3650,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,4
21521,0,18615,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,3
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,2
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,4


In [ ]:
credit['income_group'].value_counts() # проверяю равность групп по колличеству людей


1    5343
4    5342
3    5342
2    5342
Name: income_group, dtype: int64

- 1 Доход 5364 человек меньше или равен 107,6 тыс. рублей  зарабатывают меньше 75% всей выборки
- 2 доход 5479 человек находится в диапазоне от 107,6 до 142,6 тыс. рублей или больше 25% всей выборки
- 3 доход 5247 человек находится в диапазоне от 142,6 до 195,8 тыс. рублей  или больше 50% всей выборки
- 4 доход, 5364 человек зарабатывают больше 195,8 тыс. рублей или больше 75% всей выборки

### Вывод

При категоризации необходимо самостоятельно принимать решения по разбивке по категориям, исходя из доступных данных или просто разделяя на равные доли.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
print(credit['children'].value_counts())#выведу перед глазами распространенность количества детей, для удобства выводов

credit.groupby('income_group')['debt'].mean().to_frame().sort_values(by='debt') #отношение колличества детей с долей задолженности


0    14037
1     4840
2     2114
3      328
4       41
5        9
Name: children, dtype: int64


,debt
income_group,
4,0.071696
1,0.078420
2,0.084613
3,0.089667


### Вывод

Как видно из простой группировки можно проследить зависимость не возврата долга от количества детей.  Судя по имеющимся данным, наличие детей увеличивает вероятность задолженности по сравнению с отсутствием детей у клиента. Идет прямое возрастание до 4‑х детей. Заемщики с 5 детьми не имеют просрочек. Но в данном дата фрейме выборка представлена малым количеством.
Возможно задержка по кредитам - это следствие возрастающей финансовой нагрузкой с количеством детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
print(credit['family_status'].value_counts()) #выведу перед глазами распространенность частоты семейных статусов
credit.groupby('family_status')['debt'].mean().to_frame().sort_values(by='debt') #отношение семейного статуса с долей задолженности

женат / замужем          12295
гражданский брак          4141
Не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64


,debt
family_status,
вдовец / вдова,0.064990
в разводе,0.071730
женат / замужем,0.075397
гражданский брак,0.093214
Не женат / не замужем,0.097709


### Вывод

Как мы видим люди состоящие или состоявшие в официально зарегистрированном браке более дисциплинированны, чем люди не состоящие в браке или по каким либо причинам не вступающие в него. Соответственно документ о наличии супруга может быть положительно оценен при выдаче кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
credit.groupby('income_group')['debt'].mean().to_frame().sort_values(by='debt') #отношение финансовой группы с долей задолженности

,debt
income_group,
4,0.071696
1,0.078420
2,0.084613
3,0.089667


### Вывод

Клиент зарабатывает больше, чем 75% других клиентов из выборки, он с наименьшей вероятностью окажется в должниках. Но в целом частота задолженности растет вместе с размером заработной платы. Предполагаю, что такая закономерность, в следствии завышения людьми показателей реальной заработной платы для получения более крупных кредитов, впоследствии они чаще сталкиваются с проблемами выплаты более крупных сум. 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
credit.groupby('purpose_lemm')['debt'].mean().to_frame().sort_values(by='debt')


,debt
purpose_lemm,
недвижимость,0.072357
свадьба,0.079276
образование,0.092593
автомобиль,0.093327


Получаем колличество заемщиков по каждой категории

In [ ]:
all_borrower = credit.pivot_table(index=['purpose_lemm'], values='debt', aggfunc='count')

Получаем колличество должников по каждой категории

In [ ]:
dept = credit.pivot_table(index=['purpose_lemm'], values='debt', aggfunc='sum')

Получаем процент должников по каждой категории

In [ ]:
procent_debt = round(dept / all_borrower * 100, 2)
print(procent_debt.sort_values(by = 'debt', ascending = True))

              debt
purpose_lemm      
недвижимость  7.24
свадьба       7.93
образование   9.26
автомобиль    9.33


### Вывод

Самая низкая вероятность задолженности у людей берущих кредиты на недвижимость. Я думаю это связанно  с риском потери недвижимости при невыплатах кредита. Далее идут кредиты на свадьбу, как мы уже выяснили люди состоящие в браке в целом чаще выплачивают кредиты. А автомобиль является более рискованной покупкой связанной с его эксплуатацией, за счет маркетинга и статусности в глазах общественности, покупка автомобиля более импульсивное решение.

### Шаг 4. Общий вывод

Идеальный заемщик кредита это: состоящий в браке, без детей, зарабатывает больше 196 тыс. рублей в месяц, приобретатель недвижимости.
1. Наличие и количество детей повышает риск невыплат по кредиту
2. Люди официально состоящие в браке более ответственны
3. Люди с доходом выше среднего являются более безопасными клиентами, но необходимо тщательно проверять достоверность представленных документов.
4. Кредиты на недвижимость являются наиболее взвешенными, а покупка автомобилей наиболее импульсивными покупками.